In [22]:
%pylab inline
from snakebite.client import Client
import pyaml as yaml
import sys
import pythonrun
reload(pythonrun)
import os
import seaborn as sns
from sklearn import metrics
import scala_python_endive_wrapper
reload(scala_python_endive_wrapper)
from scala_python_endive_wrapper import *
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


In [9]:
hdfsclient = Client("amp-spark-master.amp", 8020, use_trash=False)
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')
logpath = "/tmp/log"

In [14]:
executor_mem = "100g"
cores_per_executor = 8
num_executors = 50
EGR1_PATH = " /data/anv/DREAMDATA/aggregated/full_dnase/EGR1"

In [18]:
# this is where the magic happens
results = [] 
res = run_kitchensink_featurize_pipeline(EGR1_PATH,
                           "/tmp/filters.csv", 
                           logpath, 
                           seed=0,
                           filter_gen_gen=make_gaussian_filter_gen,
                           cores_per_executor=cores_per_executor,
                           sample=1.0,
                           executor_mem=executor_mem,
                           num_partitions=cores_per_executor*num_executors,
                           featuresOutput="/user/vaishaal/tmp/featurizedWindows/full/replicate-{0}/".format(seed),
                           num_executors=num_executors)
results.append(res)

(256, 32)


Exception: invocation terminated with non-zero exit status

In [ ]:
chromosomes = map(lambda x: "chr{0}".format(x+1), range(23))
chromosomes.append('chrX')
chromosomes.append('chrY')
# CHANGE ME BASED ON ENUM + TF
egr1_cell_types = string_to_enum_celltypes(['GM12878', 'H1hESC', 'HCT116', 'MCF7'])

In [ ]:
results_df = cross_validate("/user/vaishaal/tmp/featurizedWindows/full/replicate-0", hdfsclient, chromosomes, egr1_cell_types)